# KRX 분기 투자지표 데이터 수집

In [1]:
import numpy as np
import pandas as pd
import nfn_crawring as nf
import gbq_service as gbq

In [2]:
df_krx = gbq.get_krx_list()
df_krx = df_krx.sort_values('market_cap', ascending=False).reset_index(drop=True)
df_krx

,stock_code,corp_name,market,department,sector_code,sector_name,reported_month,listed_date,stock_type,price,market_cap,stock_num,updated_date
0,005930,삼성전자,KOSPI,,032604,통신 및 방송 장비 제조업,12,1975-06-11,주권,85000,507431516750000,5969782550,2024-04-02
1,000660,SK하이닉스,KOSPI,,032601,반도체 제조업,12,1996-12-26,주권,186300,135626840599500,728002365,2024-04-02
2,373220,LG에너지솔루션,KOSPI,,032802,일차전지 및 축전지 제조업,12,2022-01-27,주권,393000,91962000000000,234000000,2024-04-02
3,207940,삼성바이오로직스,KOSPI,,032101,기초 의약물질 및 생물학적 제제 제조업,12,2016-11-10,주권,816000,58077984000000,71174000,2024-04-02
4,005380,현대차,KOSPI,,033001,자동차용 엔진 및 자동차 제조업,12,1974-06-28,주권,220000,46536931320000,211531506,2024-04-02
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2680,413300,티엘엔지니어링,KONEX,일반기업부,137201,"건축기술, 엔지니어링 및 관련 기술 서비스업",12,2022-09-27,주권,1949,2544448735,1305515,2024-04-02
2681,245450,씨앤에스링크,KONEX,일반기업부,032604,통신 및 방송 장비 제조업,12,2016-06-24,주권,1500,2369940000,1579960,2024-04-02
2682,248020,젬,KONEX,일반기업부,032804,전구 및 조명장치 제조업,12,2021-10-22,주권,280,1335894000,4771050,2024-04-02
2683,322190,베른,KONEX,일반기업부,033002,자동차 차체 및 트레일러 제조업,12,2020-12-29,주권,99,883594503,8925197,2024-04-02


In [3]:
# collection_list = df_krx[['stock_code', 'corp_name']].values[0:2]
collection_list = df_krx[['stock_code', 'corp_name']].values

In [4]:
df_gbq = pd.DataFrame()
df_exception = pd.DataFrame(columns=['stock_code', 'corp_name', 'err_msg'])

for index, corp in enumerate(collection_list):
    try:
        print('No. ', index)
        print(corp, '투자지표(분기) 데이터 크롤링 시작 ===>')
        # 연간 수익성 투자지표
        df_pr = nf.crawl_nfn(stock_code=corp[0], type=nf.DataType.IV, subType=nf.IVType.PROFIT, period=nf.Period.QUARTER)
        df_pr = gbq.convert_gbq_quarter(df_pr, corp[0], corp[1], 'PR')
        
        # 연간 성장성 투자지표
        df_gr = nf.crawl_nfn(stock_code=corp[0], type=nf.DataType.IV, subType=nf.IVType.GROWTH, period=nf.Period.QUARTER)
        df_gr = gbq.convert_gbq_quarter(df_gr, corp[0], corp[1], 'GR')
        
        # 연간 안정성 투자지표
        df_st = nf.crawl_nfn(stock_code=corp[0], type=nf.DataType.IV, subType=nf.IVType.STABILITY, period=nf.Period.QUARTER)
        df_st = gbq.convert_gbq_quarter(df_st, corp[0], corp[1], 'ST')
        
        # 연간 활동성 투자지표
        df_ac = nf.crawl_nfn(stock_code=corp[0], type=nf.DataType.IV, subType=nf.IVType.ACTIVITY, period=nf.Period.QUARTER)
        df_ac = gbq.convert_gbq_quarter(df_ac, corp[0], corp[1], 'AC')
        
        # 연간 가치지표 투자지표
        df_va = nf.crawl_nfn(stock_code=corp[0], type=nf.DataType.IV, subType=nf.IVType.VALUEABLE, period=nf.Period.QUARTER)
        df_va = gbq.convert_gbq_quarter(df_va, corp[0], corp[1], 'VA')
        
        # 데이터 합치기
        df_gbq = pd.concat([df_gbq, df_pr, df_gr, df_st, df_ac, df_va])
        
        print(corp, '투자지표(분기) 데이터 크롤링 종료 ===>\n\n')
        
    except Exception as e:
        err_msg = str(type(e)) + ': ' + str(e)
        print(err_msg)
        df_exception.loc[len(df_exception)] = [corp[0], corp[1], err_msg]

No.  0
['005930' '삼성전자'] 투자지표(분기) 데이터 크롤링 시작 ===>
https://navercomp.wisereport.co.kr/v2/company/cF4002.aspx?cmp_cd=005930&frq=1&rpt=1&finGubun=MAIN&frqTyp=1&cn=&encparam=aWFCL0hENG5hVkFpMGJQNkJKSThzdz09
https://navercomp.wisereport.co.kr/v2/company/cF4002.aspx?cmp_cd=005930&frq=1&rpt=2&finGubun=MAIN&frqTyp=1&cn=&encparam=SzVMNThUY2ZzV1Vtd29QYXoyU25MZz09
https://navercomp.wisereport.co.kr/v2/company/cF4002.aspx?cmp_cd=005930&frq=1&rpt=3&finGubun=MAIN&frqTyp=1&cn=&encparam=RjUzbkpwTjc3QmNJTzR6NTJDVVZBZz09
https://navercomp.wisereport.co.kr/v2/company/cF4002.aspx?cmp_cd=005930&frq=1&rpt=4&finGubun=MAIN&frqTyp=1&cn=&encparam=MHJHeHNUdXF0STF6WXdmSWJsWGc2dz09
https://navercomp.wisereport.co.kr/v2/company/cF4002.aspx?cmp_cd=005930&frq=1&rpt=5&finGubun=MAIN&frqTyp=1&cn=&encparam=d3F0NUZyejVGaUd4MDR6blVxSjBKQT09
['005930' '삼성전자'] 투자지표(분기) 데이터 크롤링 종료 ===>


No.  1
['000660' 'SK하이닉스'] 투자지표(분기) 데이터 크롤링 시작 ===>
https://navercomp.wisereport.co.kr/v2/company/cF4002.aspx?cmp_cd=000660&frq=1&rpt=1&finG

In [5]:
df_gbq.to_csv('./csv/iv_quarter_240403.csv')
df_gbq

,stock_code,corp_name,year,month,quarter,type,order,account_name,amount
id,,,,,,,,,
005930_2022_12_4_PR_000,005930,삼성전자,2022,12,4,PR,0,매출총이익률,30.990
005930_2022_12_4_PR_001,005930,삼성전자,2022,12,4,PR,1,매출총이익＜당기＞,218368.470
005930_2022_12_4_PR_002,005930,삼성전자,2022,12,4,PR,2,매출액＜당기＞,704645.750
005930_2022_12_4_PR_003,005930,삼성전자,2022,12,4,PR,3,영업이익률,6.111
005930_2022_12_4_PR_004,005930,삼성전자,2022,12,4,PR,4,영업이익＜당기＞,43061.310
...,...,...,...,...,...,...,...,...,...
308700_2023_12_4_VA_031,308700,테크엔,2023,12,4,VA,31,DPS,NaN
308700_2023_12_4_VA_032,308700,테크엔,2023,12,4,VA,32,현금DPS,NaN
308700_2023_12_4_VA_033,308700,테크엔,2023,12,4,VA,33,주식DPS,NaN


In [6]:
gbq.load_to_iv_quarter(df_gbq)

LoadJob<project=hello-phase3, location=asia-northeast3, id=5515b5d9-deb7-4878-b78c-68ed3627c4c3> 
Uploaded:  1782057


In [7]:
df_exception

,stock_code,corp_name,err_msg
0,038950,파인디지털,<class 'requests.exceptions.JSONDecodeError'>:...
1,221980,케이디켐,<class 'requests.exceptions.JSONDecodeError'>:...
2,312610,에이에프더블류,<class 'requests.exceptions.JSONDecodeError'>:...
